**AutoML OSS入門（5）**

# GUI操作による直感的なAutoMLを実現する「H2O」

本ノートブックの紹介記事と併せてご覧ください。
- ＠IT連載 AutoML OSS入門（5）- 第5回GUI操作による直感的なAutoMLを実現する「H2O」を解説

なお、本ノートブックの扱い方や使用するデータについては、連載記事の第1回を参照してください。
- [＠IT連載 AutoML OSS入門（１）- 第1回「機械学習モデル構築作業の煩雑さを解消する「AutoML」とは――歴史、動向、利用のメリットを整理する」](https://www.atmarkit.co.jp/ait/articles/2107/02/news006.html)

---
 ***注意***

***本ノートブックは、ChromeまたはFirefox、Edgeを使用することを前提としています。***
 
※Safariを使用した場合、H2O Flow（以下、Flow）へアクセスができないことが確認されています。


## タイタニックデータでAutoML

タイタニックの生存予測データを使って、H2Oを紹介していきます。

### セットアップ

Google Colab上にPython用のH2Oをインストールします。

In [ ]:
# Javaが使用できるか確認（バージョン情報が表示されれば良い）
!java --version

openjdk 11.0.11 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [ ]:
# 依存関係にあるライブラリのインストール
!pip install requests
!pip install tabulate
!pip install future

In [ ]:
# クリーンな環境でH2Oを動作させるため、既存のH2Oライブラリをアンインストール
!pip uninstall -y h2o

# Python用H2Oライブラリのインストール
!pip install h2o==3.34.0.1

     |████████████████████████████████| 175.8 MB 36 kB/s 
  Created wheel for h2o: filename=h2o-3.34.0.1-py2.py3-none-any.whl size=175823552 sha256=6283a27e37134189fa01740e0a5f81d92fab92d317c73bb9f94ce3f056f1777d
  Stored in directory: /root/.cache/pip/wheels/5c/7b/60/014a2b6d009793271276a9a41cb954659c8f23b8823ca21625
Successfully built h2o


H2Oサーバを起動するため、ライブラリのインポートを行います。

ここではFlow上でのAutoMLがメインとなるため、H2Oサーバの起動などはセットアップの一部として扱います。


In [ ]:
# H2Oライブラリのインポート
import h2o

# バージョンの確認（3.34.0.1と表示されれば良い）
print('バージョン:{}'.format(h2o.__version__))

バージョン:3.34.0.1


タイタニックの生存予測データをダウンロードします。

※データの詳細は、[第1回記事の「タイタニックの生存予測データの取得方法と解説」](https://www.atmarkit.co.jp/ait/articles/2107/02/news006.html)を参照してください。

In [ ]:
# データのダウンロード
!wget -N https://github.com/aiq2020-tw/automl-notebooks/raw/main/titanic.zip
!unzip titanic.zip

--2021-10-02 10:12:40--  https://github.com/aiq2020-tw/automl-notebooks/raw/main/titanic.zip
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aiq2020-tw/automl-notebooks/main/titanic.zip [following]
--2021-10-02 10:12:40--  https://raw.githubusercontent.com/aiq2020-tw/automl-notebooks/main/titanic.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34877 (34K) [application/zip]
Saving to: ‘titanic.zip’

titanic.zip         100%[===================>]  34.06K  --.-KB/s    in 0.004s  

Last-modified header missing -- time-stamps turned off.
2021-10-02 10:12:40 (8.70 MB/s) - ‘titanic.zip’ saved [3487

カレントディレクトリーに以下の3つのファイルが展開されました。

*   train.csv: 学習用のデータ
*   test.csv: 予測用のデータ
*   gender_submission.csv: 予測した結果をKaggleに提出するフォーマットを示すファイル

### モデルの学習

まずはH2Oサーバを立ち上げます。その後、Flowを用いてH2Oを操作し、ノーコードでのAutoMLを実行していきます。


In [ ]:
# H2Oサーバの起動
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,24 mins 15 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.1
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_unknownUser_zs83am
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.071 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
from google.colab.output import eval_js

# Flowへのアクセス用URLの取得
# 出力されたURLをクリックすることで、アクセス可能
base_url = eval_js('google.colab.kernel.proxyPort(54321)')
print('{}flow/index.html'.format(base_url))

https://v2i9e76k0c-496ff2e9c6d22116-54321-colab.googleusercontent.com/flow/index.html


**これ以降のAutoMLは、Flowを操作し、進めます。**

---


GitHub上の本ノートブックと同じ階層にflowファイルを配置しているため、ローカルにダウンロード後、Flowにアップロードすることで、記事で紹介した内容をコード形式ですぐに実行できます。

*   ダウンロードするファイル：H2O_Flow_titanic.flow
*   アップロード方法：Flowのメニューバーにおいて以下の順に操作してください。

　　　Flow → Open Flow → ファイルを選択 → flowファイルを選択 → Open
*   コード形式の実行方法：実行対象のコードが記載されたセルにおいて「Ctrl + Enter」を押下


### 予測


Flowを用いた予測結果をKaggleに提出するため、提出用に加工します。

なお、予測結果はsubmission.csvに出力済みであることを前提としています。

In [ ]:
# 予測結果を出力したファイルのインポート
pred = h2o.import_file('./submission.csv')

# 提出用列の抽出と列名の修正
submission = pred['PassengerId']
submission['Survived'] = pred['predict']

# 提出用ファイルの出力
h2o.export_file(frame=submission, path='submission.csv', force=True)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


**提出用ファイルの出力後、ポートを開放するためにH2Oサーバをシャットダウンします。**

ポート開放を行わない場合、以降のコードが正しく実行されない可能性があるため、必ずH2Oサーバをシャットダウンしてください。

In [ ]:
# H2Oサーバのシャットダウン
h2o.shutdown()

H2O session _sid_9cf3 closed.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  


## その他の機能と応用

### JavaコマンドによるH2Oサーバの起動

##### セットアップ

jarファイルを取得し、JavaコマンドでH2Oサーバを起動します。

In [ ]:
# jarファイルを含む圧縮ファイルのダウンロードおよび解凍
h2o_version = 'h2o-3.34.0.1'
!wget -N https://h2o-release.s3.amazonaws.com/h2o/rel-zizler/1/$h2o_version'.zip'
!unzip ./$h2o_version'.zip'

--2021-10-02 10:33:03--  https://h2o-release.s3.amazonaws.com/h2o/rel-zizler/1/h2o-3.34.0.1.zip
Resolving h2o-release.s3.amazonaws.com (h2o-release.s3.amazonaws.com)... 52.217.174.217
Connecting to h2o-release.s3.amazonaws.com (h2o-release.s3.amazonaws.com)|52.217.174.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 531017426 (506M) [application/zip]
Saving to: ‘h2o-3.34.0.1.zip’

h2o-3.34.0.1.zip    100%[===================>] 506.42M  15.4MB/s    in 35s     

2021-10-02 10:33:38 (14.6 MB/s) - ‘h2o-3.34.0.1.zip’ saved [531017426/531017426]

Archive:  ./h2o-3.34.0.1.zip
   creating: h2o-3.34.0.1/
   creating: h2o-3.34.0.1/R/
  inflating: h2o-3.34.0.1/R/h2o_client_3.34.0.1.tar.gz  
  inflating: h2o-3.34.0.1/R/README.txt  
  inflating: h2o-3.34.0.1/R/h2o_3.34.0.1.tar.gz  
   creating: h2o-3.34.0.1/python/
  inflating: h2o-3.34.0.1/python/h2o-3.34.0.1-py2.py3-none-any.whl  
  inflating: h2o-3.34.0.1/python/h2o_client-3.34.0.1-py2.py3-none-any.whl  
  inflating:

In [ ]:
import subprocess
import re

# H2Oサーバを別プロセスで起動
start_h2o_cmd = 'java -jar ./'+h2o_version+'/h2o.jar'
proc = subprocess.Popen('exec ' + start_h2o_cmd, stdout=subprocess.PIPE, shell=True)

# 起動ログを最大100行監視し、正常に起動することを確認
for i in range(100):
    line = proc.stdout.readline()
    print(line)
    # Flowのアクセス用URLが出力されたら起動完了のため停止
    if ('Open H2O Flow in your web browser:' in str(line)):
        break

b'10:34:43.045 [main] INFO  hex.tree.xgboost.util.NativeLibrary - Loaded library from lib/linux_64/libxgboost4j_gpu.so (/tmp/libxgboost4j_gpu12639627023244453577.so)\n'
b'10-02 10:34:43.576 172.28.0.2:54321      938          main  INFO water.default: ----- H2O started  -----\n'
b'10-02 10:34:43.577 172.28.0.2:54321      938          main  INFO water.default: Build git branch: rel-zizler\n'
b'10-02 10:34:43.578 172.28.0.2:54321      938          main  INFO water.default: Build git hash: 995acba1cf2b5c321251cb0d5dca4cbc45c6a3d9\n'
b'10-02 10:34:43.578 172.28.0.2:54321      938          main  INFO water.default: Build git describe: jenkins-master-5565-16-g995acba\n'
b'10-02 10:34:43.579 172.28.0.2:54321      938          main  INFO water.default: Build project version: 3.34.0.1\n'
b'10-02 10:34:43.580 172.28.0.2:54321      938          main  INFO water.default: Build age: 17 days\n'
b"10-02 10:34:43.580 172.28.0.2:54321      938          main  INFO water.default: Built by: 'jenkins'\n"
b"

In [ ]:
# Flowへのアクセス用URLの取得
# 出力されたURLをクリックすることで、アクセス可能
from google.colab.output import eval_js

base_url = eval_js('google.colab.kernel.proxyPort(54321)')
print('{}flow/index.html'.format(base_url))

https://ep4qyl7t9or-496ff2e9c6d22116-54321-colab.googleusercontent.com/flow/index.html


In [ ]:
# Flowへアクセスできることを確認後、H2Oサーバのプロセスを停止（シャットダウン）
proc.kill()

### Pythonを用いたコーディングによるAutoML

##### セットアップ

AutoMLに必要となるライブラリをインポート後、H2Oサーバを起動します。

In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
# H2Oサーバの起動
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp7oifg93w
  JVM stdout: /tmp/tmp7oifg93w/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp7oifg93w/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.1
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_unknownUser_zs83am
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


##### データのロード

タイタニックの学習および予測用データをロード後、不要な列の削除などの処理を行います。

In [ ]:
# 学習・予測用データのロード
X_train = h2o.import_file('train.csv')
X_test = h2o.import_file('test.csv')

# 目的変数と説明変数のカラム名を保存
X_col = X_train.columns
y_col = 'Survived'
X_col.remove(y_col)

# 生存予測は二値分類となるため、カテゴリデータに設定
X_train[y_col] = X_train[y_col].asfactor()

# 不要な列を説明変数から削除
ignore_col = ['PassengerId', 'Name']
for col in ignore_col:
    X_col.remove(col)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


##### モデルの学習

In [ ]:
# モデルの学習開始
automl = H2OAutoML(seed=42, max_runtime_secs=60*3)
automl.train(x=X_col, y=y_col, training_frame=X_train)

# 予測精度が高い順にモデルを表示
lb = automl.leaderboard
lb.head(rows=lb.nrows)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_4_AutoML_1_20211002_104358,0.88257,0.392844,0.865931,0.176373,0.347572,0.120806
GBM_5_AutoML_1_20211002_104358,0.879526,0.403105,0.859847,0.179153,0.354466,0.125646
StackedEnsemble_BestOfFamily_3_AutoML_1_20211002_104358,0.878857,0.39795,0.863471,0.174288,0.349327,0.122029
StackedEnsemble_AllModels_1_AutoML_1_20211002_104358,0.877973,0.398255,0.863103,0.17412,0.348821,0.121676
StackedEnsemble_AllModels_2_AutoML_1_20211002_104358,0.877262,0.400099,0.861409,0.174791,0.349978,0.122485
XRT_1_AutoML_1_20211002_104358,0.877174,0.422415,0.849412,0.182485,0.363095,0.131838
StackedEnsemble_BestOfFamily_2_AutoML_1_20211002_104358,0.876903,0.400642,0.86217,0.173857,0.350041,0.122529
GBM_grid_1_AutoML_1_20211002_104358_model_13,0.875598,0.412925,0.859932,0.184618,0.356825,0.127324
GBM_grid_1_AutoML_1_20211002_104358_model_32,0.873928,0.408319,0.858603,0.176613,0.355536,0.126406
GBM_3_AutoML_1_20211002_104358,0.873784,0.407812,0.856749,0.173881,0.355408,0.126315


##### 予測

In [ ]:
# 予測精度が最も高いモデルで予測
preds = automl.leader.predict(X_test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.7/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'Cabin' has levels not trained on: ["A11", "A18", "A21", "A29", "A9", "B10", "B11", "B24", "B26", "B36", ...19 not listed..., "D38", "D40", "D43", "E39 E41", "E45", "E52", "E60", "F", "F E46", "F E57"]
  warnings.warn(w)


In [ ]:
# Kaggleに提出するため、予測結果を提出用に加工
submit_df = preds.as_data_frame()
submit_df['PassengerId'] = X_test.as_data_frame()['PassengerId']

# 提出用列の抽出と列名の修正
submit_df = submit_df[['PassengerId', 'predict']]
submit_df = submit_df.rename(columns={'predict': 'Survived'})

submit_df.to_csv('submission_python_code.csv', index=False)

### Javaアプリケーションへの組み込み方法

#### 構築したモデルの出力

In [ ]:
# 前段で構築した予測精度が最も高いモデルをMOJO形式で出力
model_file = automl.leader.download_mojo()

# モデル名とパスを表示
# 表示されたモデルのファイル名をこの後作成するMain.javaで指定
print(model_file)

/content/StackedEnsemble_BestOfFamily_4_AutoML_1_20211002_104358.zip


#### javaファイルの作成

In [ ]:
# javaファイルを出力
%%writefile Main.java
import hex.genmodel.MojoModel;
import hex.genmodel.easy.EasyPredictModelWrapper;
import hex.genmodel.easy.RowData;
import hex.genmodel.easy.exception.PredictException;
import hex.genmodel.easy.prediction.BinomialModelPrediction;

public class Main {

  public static void main(String[] args) throws Exception {
    try {
      // モデルの読み込み(出力したモデルのファイル名を拡張子込みで指定）
      EasyPredictModelWrapper model = new EasyPredictModelWrapper(MojoModel.load(args[0]));

      //データのロード
      RowData titanicData = new RowData();
      titanicData.put("PassengerId", "4");
      titanicData.put("Pclass", "1");
      titanicData.put("Name", "Futrelle, Mrs. Jacques Heath (Lily May Peel)");
      titanicData.put("Sex", "female");
      titanicData.put("Age", "35");
      titanicData.put("SibSp", "1");
      titanicData.put("Parch", "0");
      titanicData.put("Ticket", "113803");
      titanicData.put("Fare", "53.1");
      titanicData.put("Cabin", "C123");
      titanicData.put("Embarked", "S");

      String isSurvived = "1";

      // 予測
      BinomialModelPrediction pred = model.predictBinomial(titanicData);

      System.out.println("＝＝＝予測結果＝＝＝");
      System.out.println("正解: " + isSurvived);
      System.out.println("予測: " + pred.label);
      System.out.println("----");
      System.out.println("各クラスの所属確率");
      for (int i = 0; i < pred.classProbabilities.length; i++) {
        System.out.println(
          String.valueOf(i) + ": " + pred.classProbabilities[i]
        );
      }
    } catch (PredictException e) {
      e.printStackTrace();
    }
  }
}

Overwriting Main.java


#### javaファイルのコンパイルと実行

In [ ]:
# コンパイル時に参照するためのライブラリを取得
!curl http://localhost:54321/3/h2o-genmodel.jar > h2o-genmodel.jar

# javaファイルをコンパイル
!javac -cp h2o-genmodel.jar -J-Xms2g Main.java

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3809k  100 3809k    0     0   116M      0 --:--:-- --:--:-- --:--:--  116M


In [ ]:
# 実行
!java -cp .:./h2o-genmodel.jar Main $model_file

＝＝＝予測結果＝＝＝
正解: 1
予測: 1
----
各クラスの所属確率
0: 0.0752803670332216
1: 0.9247196329667784


以上で、タイタニックの生存予測データを使ったH2Oの紹介は終了です。